### Get the Personalize boto3 Client

In [2]:
import boto3

import json
import numpy as np
import pandas as pd
import time

personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')
iam = boto3.client("iam")
s3 = boto3.client("s3")

### Specify a Bucket and Data Output Location

In [2]:
bucket = "personalize-demo"       # replace with the name of your S3 bucket
filename = "movie-lens-100k.csv"  # replace with a name that you want to save the dataset under

### Download, Prepare, and Upload Training Data

#### Download and Explore the Dataset

In [8]:
!wget -N http://files.grouplens.org/datasets/movielens/ml-100k.zip
!unzip -o ml-100k.zip
data = pd.read_csv('./ml-100k/u.data', sep='\t', names=['USER_ID', 'ITEM_ID', 'RATING', 'TIMESTAMP'])
pd.set_option('display.max_rows', 5)
data

--2020-05-05 09:38:29--  http://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 304 Not Modified
File ‘ml-100k.zip’ not modified on server. Omitting download.

Archive:  ml-100k.zip
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.base         
  inflating: ml-100k/u2.test         
  inflating: ml-100k/u3.base         
  inflating: ml-100k/u3.test         
  inflating: ml-100k/u4.base         
  inflat

,USER_ID,ITEM_ID,RATING,TIMESTAMP
0,196,242,3,881250949
1,186,302,3,891717742
...,...,...,...,...
99998,13,225,2,882399156
99999,12,203,3,879959583


#### Optional security practice: Protect data at rest - Encrypt/decrypt your dataset
We are using a pre-made dataset that hasn't been encrypted so there is no need to decrypt this dataset. However, it would be a good security practice to store your datasets encrypted.

For more information on encrypting your data when using S3, visit https://docs.aws.amazon.com/AmazonS3/latest/dev/KMSUsingRESTAPI.html

#### Optional security practice: Protect data in transit - SSL access only for S3 bucket

In [9]:
requires_ssl_access_policy = {
    "Version": "2012-10-17",
    "Id": "RequireSSLAccess",
    "Statement": [
        {
            "Sid": "RequireSSLAccess",
            "Effect": "Deny",
            "Principal": "*",
            "Action": "*",
            "Resource": [
                "arn:aws:s3:::{}".format(bucket),
                "arn:aws:s3:::{}/*".format(bucket)
            ],
            "Condition": {
                "Bool": {
                    "aws:SecureTransport": "false"
                }
            }
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket, Policy=json.dumps(requires_ssl_access_policy))

{'ResponseMetadata': {'RequestId': 'FA2BA6A24A738415',
  'HostId': 'PWU2HsPCLALBFzYHzVEUK5EuODQkMTrj2l9IuCs3x+GBDlM24tg2k4BF1fMlnmzCAwy/KRVplQw=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'PWU2HsPCLALBFzYHzVEUK5EuODQkMTrj2l9IuCs3x+GBDlM24tg2k4BF1fMlnmzCAwy/KRVplQw=',
   'x-amz-request-id': 'FA2BA6A24A738415',
   'date': 'Tue, 05 May 2020 16:38:34 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

#### Additional security note:
Some users prevent accidental information disclosure by limiting S3 access to only come from a VPC. Another common security practice is to validate this limited access. It should be noted that this security check will fail when performed against S3 buckets used for Personalize - as Personalize copies data from the user's S3 into the internal systems used by Personalize (during dataset import jobs).

#### Optional security practice: validate bucket owner matches your account canonical id
[More information about canonical ids here](https://docs.aws.amazon.com/general/latest/gr/acct-identifiers.html#FindingCanonicalId)

In [9]:
bucket_owner_id = boto3.client('s3').get_bucket_acl(Bucket=bucket)['Owner']['ID']
print("This bucket belongs to: {} ".format(bucket_owner_id))

This bucket belongs to: 28398dc6b1acac01a4a73b246b5f9c9a688f50b9ce70240f74c0f90ebf5e2c61 


#### Optional security practice: Protect data integrity - Enable S3 bucket versioning


In [10]:
s3_resource = boto3.resource('s3')
bucket_versioning = s3_resource.BucketVersioning(bucket)
bucket_versioning.enable()

{'ResponseMetadata': {'RequestId': '4C3AA32EF96CA240',
  'HostId': 'HQaKmMvUTbXTRsxLL/9BubrfD09xisEO8x72cpQP0syhAH9dXKRx7gqmzchq3TbERdwprUKcRmQ=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'HQaKmMvUTbXTRsxLL/9BubrfD09xisEO8x72cpQP0syhAH9dXKRx7gqmzchq3TbERdwprUKcRmQ=',
   'x-amz-request-id': '4C3AA32EF96CA240',
   'date': 'Mon, 04 May 2020 20:46:23 GMT',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

#### Prepare and Upload Data

In [ ]:
data = data[data['RATING'] > 3.6]                # keep only movies rated 3.6 and above
data = data[['USER_ID', 'ITEM_ID', 'TIMESTAMP']] # select columns that match the columns in the schema below
data.to_csv(filename, index=False)

boto3.Session().resource('s3').Bucket(bucket).Object(filename).upload_file(filename)

### Create Schema

In [ ]:
schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        }
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = "DEMO-schema",
    schema = json.dumps(schema)
)

schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

{
  "schemaArn": "arn:aws:personalize:us-west-2:237539672711:schema/DEMO-schema", 
  "ResponseMetadata": {
    "RetryAttempts": 0, 
    "HTTPStatusCode": 200, 
    "RequestId": "12eb7cba-2b64-4be9-9f6e-eeebff7629a5", 
    "HTTPHeaders": {
      "date": "Tue, 04 Dec 2018 05:49:04 GMT", 
      "x-amzn-requestid": "12eb7cba-2b64-4be9-9f6e-eeebff7629a5", 
      "content-length": "79", 
      "content-type": "application/x-amz-json-1.1", 
      "connection": "keep-alive"
    }
  }
}


#### Optional security practice - Protect data at rest - Encrypt datasets under Personalize
If you skip this step, do not pass kmsKeyArn or roleArn when you create your dataset group.

In [3]:
kmsKeyArn = boto3.client('kms').create_key(Description="personalize-data")['KeyMetadata']['Arn']
print(kmsKeyArn)
key_accessor_policy_name = "AccessPersonalizeDatasetPolicy"
key_accessor_policy = {
  "Version": "2012-10-17",
  "Statement": {
    "Effect": "Allow",
    "Action": [
      "kms:*"
    ],
    "Resource": [ kmsKeyArn ]
  }
}
key_access_policy = iam.create_policy(
    PolicyName = key_accessor_policy_name,
    PolicyDocument = json.dumps(key_accessor_policy)
)

key_access_role_name = "AccessPersonalizeDatasetRole"
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

create_role_response = iam.create_role(
    RoleName = key_access_role_name,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
)
iam.attach_role_policy(
    RoleName = create_role_response["Role"]["RoleName"],
    PolicyArn = key_access_policy["Policy"]["Arn"]
)

keyAccessRoleArn = create_role_response["Role"]["Arn"]
print(keyAccessRoleArn)

arn:aws:kms:us-west-2:001513653716:key/f5be82af-a160-4c49-813e-e5448fa95693
arn:aws:iam::001513653716:role/AccessPersonalizeDatasetRole


### Create and Wait for Encrypted Dataset Group

#### Create Encrypted Dataset Group
If you did not create a KMS Key and IAM role from the last step, then do not pass in roleArn or kmsKeyArn.

In [5]:
create_dataset_group_response = personalize.create_dataset_group(
    name = "DEMO-dataset-group",
    roleArn = keyAccessRoleArn,
    kmsKeyArn = kmsKeyArn
)

dataset_group_arn = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

ClientError: An error occurred (AccessDeniedException) when calling the CreateDatasetGroup operation: Cross-account pass role is not allowed.

#### Wait for Dataset Group to Have ACTIVE Status

In [42]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

DatasetGroup: CREATE PENDING
DatasetGroup: CREATE FAILED


### Create Dataset

In [ ]:
dataset_type = "INTERACTIONS"
create_dataset_response = personalize.create_dataset(
    name = "DEMO-dataset",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = schema_arn
)

dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

{
  "ResponseMetadata": {
    "RetryAttempts": 0, 
    "HTTPStatusCode": 200, 
    "RequestId": "29ab75c8-df6e-4807-943f-1b48014181d1", 
    "HTTPHeaders": {
      "date": "Tue, 04 Dec 2018 05:50:19 GMT", 
      "x-amzn-requestid": "29ab75c8-df6e-4807-943f-1b48014181d1", 
      "content-length": "101", 
      "content-type": "application/x-amz-json-1.1", 
      "connection": "keep-alive"
    }
  }, 
  "datasetArn": "arn:aws:personalize:us-west-2:237539672711:dataset/DEMO-dataset-group/INTERACTIONS"
}


### Prepare, Create, and Wait for Dataset Import Job

#### Attach Policy to S3 Bucket

In [ ]:
policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket),
                "arn:aws:s3:::{}/*".format(bucket)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket, Policy=json.dumps(policy))

#### Create Personalize Role

In [24]:
role_name = "PersonalizeRole"
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

create_role_response = iam.create_role(
    RoleName = role_name,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
)

# AmazonPersonalizeFullAccess provides access to any S3 bucket with a name that includes "personalize" or "Personalize" 
# if you would like to use a bucket with a different name, please consider creating and attaching a new policy
# that provides read access to your bucket or attaching the AmazonS3ReadOnlyAccess policy to the role
policy_arn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
iam.attach_role_policy(
    RoleName = role_name,
    PolicyArn = policy_arn
)

time.sleep(60) # wait for a minute to allow IAM role policy attachment to propagate

role_arn = create_role_response["Role"]["Arn"]
print(role_arn)

arn:aws:iam::660166145966:role/PersonalizeRole2


#### Create Dataset Import Job

In [ ]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "DEMO-dataset-import-job",
    datasetArn = dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket, filename)
    },
    roleArn = role_arn
)

dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:us-west-2:237539672711:dataset-import-job/DEMO-dataset-import-job", 
  "ResponseMetadata": {
    "RetryAttempts": 0, 
    "HTTPStatusCode": 200, 
    "RequestId": "3c77fe8d-d9fe-4ca5-ad03-b18e937acbb3", 
    "HTTPHeaders": {
      "date": "Tue, 04 Dec 2018 05:50:55 GMT", 
      "x-amzn-requestid": "3c77fe8d-d9fe-4ca5-ad03-b18e937acbb3", 
      "content-length": "113", 
      "content-type": "application/x-amz-json-1.1", 
      "connection": "keep-alive"
    }
  }
}


#### Wait for Dataset Import Job to Have ACTIVE Status

In [ ]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("DatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

DatasetImportJob: CREATE PENDING
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: ACTIVE


### Select Recipe

In [ ]:
list_recipes_response = personalize.list_recipes()
recipe_arn = "arn:aws:personalize:::recipe/aws-hrnn" # aws-hrnn selected for demo purposes
list_recipes_response

{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
   'content-length': '1287',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Fri, 22 Mar 2019 19:28:37 GMT',
   'x-amzn-requestid': '275695de-45b2-4077-82d4-c961ceeaf367'},
  'HTTPStatusCode': 200,
  'RequestId': '275695de-45b2-4077-82d4-c961ceeaf367',
  'RetryAttempts': 0},
 u'recipes': [{u'creationDateTime': datetime.datetime(2018, 11, 25, 16, 0, tzinfo=tzlocal()),
   u'lastUpdatedDateTime': datetime.datetime(1969, 12, 31, 16, 0, tzinfo=tzlocal()),
   u'name': u'aws-hrnn',
   u'recipeArn': u'arn:aws:personalize:::recipe/aws-hrnn',
   u'status': u'ACTIVE'},
  {u'creationDateTime': datetime.datetime(2018, 11, 25, 16, 0, tzinfo=tzlocal()),
   u'lastUpdatedDateTime': datetime.datetime(1969, 12, 31, 16, 0, tzinfo=tzlocal()),
   u'name': u'aws-hrnn-coldstart',
   u'recipeArn': u'arn:aws:personalize:::recipe/aws-hrnn-coldstart',
   u'status': u'ACTIVE'},
  {u'creationDateTime': datetime.datetime(2018, 11, 25, 16,

### Create and Wait for Solution

#### Create Solution

In [ ]:
create_solution_response = personalize.create_solution(
    name = "DEMO-solution",
    datasetGroupArn = dataset_group_arn,
    recipeArn = recipe_arn
)

solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:us-west-2:237539672711:solution/DEMO-solution", 
  "ResponseMetadata": {
    "RetryAttempts": 0, 
    "HTTPStatusCode": 200, 
    "RequestId": "2042832f-0775-43e2-86de-53a061be1f63", 
    "HTTPHeaders": {
      "date": "Mon, 03 Dec 2018 23:55:17 GMT", 
      "x-amzn-requestid": "2042832f-0775-43e2-86de-53a061be1f63", 
      "content-length": "83", 
      "content-type": "application/x-amz-json-1.1", 
      "connection": "keep-alive"
    }
  }
}


#### Create Solution Version

In [ ]:
create_solution_version_response = personalize.create_solution_version(
    solutionArn = solution_arn
)

solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-west-2:237539672711:solution/DEMO-solution/702e0792", 
  "ResponseMetadata": {
    "RetryAttempts": 0, 
    "HTTPStatusCode": 200, 
    "RequestId": "2042832f-0775-43e2-86de-53a061be1f65", 
    "HTTPHeaders": {
      "date": "Mon, 03 Dec 2018 23:55:17 GMT", 
      "x-amzn-requestid": "2042832f-0775-43e2-86de-53a061be1f65", 
      "content-length": "90", 
      "content-type": "application/x-amz-json-1.1", 
      "connection": "keep-alive"
    }
  }
}


#### Wait for Solution Version to Have ACTIVE Status

In [ ]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = solution_version_arn
    )
    status = describe_solution_version_response["solutionVersion"]["status"]
    print("SolutionVersion: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

SolutionVersion: CREATE PENDING
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGR

#### Get Metrics of Solution

In [ ]:
get_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = solution_version_arn
)

print(json.dumps(get_solution_metrics_response, indent=2))

{
  "metrics": {
    "coverage": 0.2603, 
    "mean_reciprocal_rank_at_25": 0.0539, 
    "normalized_discounted_cumulative_gain_at_5": 0.0486, 
    "normalized_discounted_cumulative_gain_at_10": 0.0649, 
    "normalized_discounted_cumulative_gain_at_25": 0.0918, 
    "precision_at_5": 0.0109, 
    "precision_at_10": 0.0098, 
    "precision_at_25": 0.0083, 
  }, 
  "solutionVersionArn": "arn:aws:personalize:us-west-2:237539672711:solution/DEMO-solution/702e0792", 
  "ResponseMetadata": {
    "RetryAttempts": 0, 
    "HTTPStatusCode": 200, 
    "RequestId": "5b5f4f4f-5249-4c0e-9f83-45e3fe22f09f", 
    "HTTPHeaders": {
      "date": "Tue, 04 Dec 2018 00:53:54 GMT", 
      "x-amzn-requestid": "5b5f4f4f-5249-4c0e-9f83-45e3fe22f09f", 
      "content-length": "724", 
      "content-type": "application/x-amz-json-1.1", 
      "connection": "keep-alive"
    }
  }
}


### Create and Wait for Campaign

#### Create Campaign

In [ ]:
create_campaign_response = personalize.create_campaign(
    name = "DEMO-campaign",
    solutionVersionArn = solution_version_arn,
    minProvisionedTPS = 1
)

campaign_arn = create_campaign_response['campaignArn']
print(json.dumps(create_campaign_response, indent=2))

{
  "campaignArn": "arn:aws:personalize:us-west-2:237539672711:campaign/DEMO-campaign", 
  "ResponseMetadata": {
    "RetryAttempts": 0, 
    "HTTPStatusCode": 200, 
    "RequestId": "527e97ba-683c-4dc7-8218-00716f22c904", 
    "HTTPHeaders": {
      "date": "Tue, 04 Dec 2018 00:54:17 GMT", 
      "x-amzn-requestid": "527e97ba-683c-4dc7-8218-00716f22c904", 
      "content-length": "83", 
      "content-type": "application/x-amz-json-1.1", 
      "connection": "keep-alive"
    }
  }
}


#### Wait for Campaign to Have ACTIVE Status

In [ ]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = campaign_arn
    )
    status = describe_campaign_response["campaign"]["status"]
    print("Campaign: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

Campaign: CREATE PENDING
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: ACTIVE


### Get Recommendations

#### Select a User and an Item

In [ ]:
items = pd.read_csv('./ml-100k/u.item', sep='|', usecols=[0,1], encoding='latin-1')
items.columns = ['ITEM_ID', 'TITLE']

user_id, item_id, _ = data.sample().values[0]
item_title = items.loc[items['ITEM_ID'] == item_id].values[0][-1]
print("USER: {}".format(user_id))
print("ITEM: {}".format(item_title))

items

USER: 711
ITEM: Silence of the Lambs, The (1991)


,ITEM_ID,TITLE
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
...,...,...
1680,1681,You So Crazy (1994)
1681,1682,Scream of Stone (Schrei aus Stein) (1991)


#### Call GetRecommendations

In [ ]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = campaign_arn,
    userId = str(user_id),
    itemId = str(item_id)
)

item_list = get_recommendations_response['itemList']
title_list = [items.loc[items['ITEM_ID'] == np.int(item['itemId'])].values[0][-1] for item in item_list]

print("Recommendations: {}".format(json.dumps(title_list, indent=2)))

Recommendations: [
  "Godfather, The (1972)", 
  "Contact (1997)", 
  "Titanic (1997)", 
  "Star Wars (1977)", 
  "Fargo (1996)", 
  "Liar Liar (1997)", 
  "Evita (1996)", 
  "Jerry Maguire (1996)", 
  "Scream (1996)", 
  "Devil's Advocate, The (1997)", 
  "Full Monty, The (1997)", 
  "Conspiracy Theory (1997)", 
  "Edge, The (1997)", 
  "Sense and Sensibility (1995)", 
  "English Patient, The (1996)", 
  "Twelve Monkeys (1995)", 
  "L.A. Confidential (1997)", 
  "As Good As It Gets (1997)", 
  "In & Out (1997)", 
  "Rock, The (1996)", 
  "Return of the Jedi (1983)", 
  "Amistad (1997)", 
  "Men in Black (1997)", 
  "Truth About Cats & Dogs, The (1996)", 
  "Alien: Resurrection (1997)"
]
